# 🤖 OpenManus MVP

> **Goal**: OSS AI エージェント (Manus クローン) の動作確認
>
> **Runtime**: CPU or T4 GPU
>
> **Time**: ~20分

## 1. インストール

In [ ]:
# OpenManus リポジトリ
!git clone https://github.com/mannaandpoem/OpenManus.git
%cd OpenManus

In [ ]:
# 依存関係
!pip install -q -r requirements.txt

## 2. 設定

In [ ]:
import os
from google.colab import userdata

# API キー設定 (Colab Secrets から)
# 事前に Secrets に OPENAI_API_KEY を登録
try:
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    print("✅ API key loaded from Secrets")
except:
    print("⚠️ Set OPENAI_API_KEY manually:")
    os.environ["OPENAI_API_KEY"] = input("Enter OpenAI API Key: ")

In [ ]:
# config.toml 作成
config = """
[llm]
model = "gpt-4o-mini"
base_url = "https://api.openai.com/v1"
api_key = "${OPENAI_API_KEY}"
max_tokens = 4096
temperature = 0.0
"""

with open("config/config.toml", "w") as f:
    f.write(config.replace("${OPENAI_API_KEY}", os.environ["OPENAI_API_KEY"]))
print("✅ Config saved")

## 3. エージェント実行

In [ ]:
import asyncio
from app.agent.manus import Manus

async def run_agent(task: str):
    agent = Manus()
    result = await agent.run(task)
    return result

In [ ]:
# テストタスク
TASK = """以下の情報を調査してまとめてください：

1. Activation Steering の最新研究 (2025-2026)
2. 主要な実装ライブラリ
3. LLM への応用事例

結果を箇条書きで出力してください。
"""

result = await run_agent(TASK)
print(result)

## 4. Hegemonikón 統合テスト

In [ ]:
# CCL スタイルのタスク
CCL_TASK = """/sop+ を実行してください。

テーマ: 「Activation Steering と SEL の統合可能性」

以下を調査：
- 技術的実現性
- 期待効果
- リスク

Perplexity スタイルで出力（引用付き）。
"""

ccl_result = await run_agent(CCL_TASK)
print(ccl_result)

## 5. 評価

In [ ]:
# OpenManus の能力評価
capabilities = {
    "Web検索": "search" in str(result).lower() or "調査" in str(result),
    "構造化出力": "1." in str(result) or "-" in str(result),
    "日本語対応": any(ord(c) > 127 for c in str(result)),
    "タスク完了": len(str(result)) > 100,
}

print("=" * 40)
print("OpenManus 能力評価")
print("=" * 40)
for cap, passed in capabilities.items():
    status = "✅" if passed else "❌"
    print(f"{status} {cap}")

---
*Hegemonikón OpenManus MVP — 2026-02-01*